In [41]:
import pandas as pd
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings("ignore")

In [231]:
# 全部的PCODE表（CAP提供）
pcode = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\產品規格相關\BPCOM\BPCOM_20230103.xlsx")
pcode = pcode[["PrCode", "GRP", "Section", "Rim", "BP", "Product Name"]]

In [237]:
# 全部的Premium list
premium = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\KPI\2023 GKPI\20230710_GKPI BSFC_June 2023.xlsx", sheet_name="Premium Products - BPCs")
premium = premium.iloc[1:]
premium = premium[["BPC", "Premium"]]

In [240]:
# 產品總表
pro_df = pd.merge(pcode, premium, how="left", left_on="PrCode", right_on="BPC").drop("BPC", axis=1)

In [72]:
# 整理實績和下半年RT3
# 實績整理

act = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2023ACT\20230710_SizeData.xlsx", sheet_name="Sheet1")

In [73]:
act = act[(act["預算/前實績"] == "23ACT") & (act["通路明細"] != "LSCM") & (act["客戶名稱"] != "普利司通")]

In [75]:
act.groupby(["通路明細"])[["條數"]].sum()

,條數
通路明細,
AG,186
CD,515
FLT-BSSC,6502
FLT-REP,1598
FT,4857
MM,0
OT,2
中盤,988
加油站,10


In [105]:
channel_keys = ['FLT-REP', 'CD', 'FLT-BSSC', 'OT', '中盤', '加油站', 'AG', '輪胎行', 'FT', 'MM', 'BSSC', 'BSSC-FLT']
channel_values = ['FLT-REP', 'T/S', 'BSSC-FLT', 'T/S', 'T/S', 'T/S', 'T/S', 'T/S', 'FLT-REP', 'T/S', 'T/S', 'BSSC-FLT']
channel_dict = dict(zip(channel_keys, channel_values))

In [81]:
act["通路明細"] = act["通路明細"].map(channel_dict)

In [89]:
act_result = act.groupby([pd.Grouper(key="整理後的日期", freq="MS"), "商品代號", "通路明細"])[["條數"]].sum().reset_index()

In [179]:
act_result["整理後的日期"] = act_result["整理後的日期"].dt.month_name()

In [92]:
## RBC3整理
rba = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\價格相關\預算\2023\20230801_RBAC3_final.xlsx")

In [111]:
rba["整理後的日期"] = pd.to_datetime(rba["年月"], format="%Y%m")

In [112]:
rba = rba[(rba["國內市場財別"] == "TBR(G)") & (rba["預算/前實績"] == "23RBAC3") & (rba["大通路分類"] != "OE") & (rba["通路明細"] != "LSCM") & (rba["客戶名稱"] != "普利司通") & (rba["整理後的日期"] >= "2023-07-01")]

In [118]:
rba_result = rba.groupby([pd.Grouper(key="整理後的日期", freq="MS"), "商品代號", "通路明細"])[["條數"]].sum().reset_index()

In [121]:
rba_result["條數"] = rba_result["條數"].astype(int)

In [177]:
rba_result["整理後的日期"] = rba_result["整理後的日期"].dt.month_name()

In [178]:
rba_result

,整理後的日期,商品代號,通路明細,條數
0,July,LSR01180,AG,12
1,July,LSR01180,BSSC,334
2,July,LSR01180,BSSC-FLT,4
3,July,LSR01270,AG,10
4,July,LSR01270,BSSC,386
...,...,...,...,...
535,December,TBR0J781,BSSC-FLT,26
536,December,TBR0J795,AG,24
537,December,TBR0J795,BSSC,792
538,December,TBR0J795,BSSC-FLT,140


In [157]:
## 整理業務填回的數據
ob = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\OB\24OB\24OB_業務填寫表格.xlsx", None)

In [158]:
dfs = []
columns = ["PCODE",	"中計", "上市時間",	"GRP", "大通路", "客戶通路", "規格", "吋別", "花紋", "產地", "Product Name", "BSCAP definition", "1月",	"2月", "3月", "4月", "5月",	"6月", "7月", "8月", "9月",	"10月",	"11月",	"12月",	"2025年", "2026年"]
for sheet in ob.keys():
    if sheet not in ['歷史數據樞紐', 'Raw data', 'Demand_Sales', 'TBR premium定義', '總表', '24OB表格']:
        df = ob[sheet].iloc[5:]
        df.columns = columns
        dfs.append(df)

In [159]:
ob_result = pd.concat(dfs)

In [160]:
ob_result = ob_result[['PCODE', '客戶通路', '1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月', '2025年', '2026年']]

In [164]:
ob_result["客戶通路"] = ob_result["客戶通路"].map(lambda x: "T/S" if x in ["FMC", "其他"] else x)

In [169]:
ob_result = ob_result.groupby(['PCODE', '客戶通路']).sum().stack().reset_index(name="條數")

In [171]:
ob_result.columns = ["商品代號", "通路明細", "整理後的日期", "條數"]

In [180]:
ob_result = ob_result[["整理後的日期", "商品代號", "通路明細", "條數"]]

In [202]:
# 合併表格
final = pd.concat([act_result, rba_result, ob_result], ignore_index=True)

In [213]:
final = final.set_index(["商品代號", "通路明細", "整理後的日期"]).unstack("整理後的日期", fill_value=0).droplevel(level=0, axis=1).reset_index()

In [226]:
final = final.rename_axis(None, axis=1)

In [229]:
final = final[['商品代號', '通路明細', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', '1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月', '2025年', '2026年']]

In [230]:
final

,商品代號,通路明細,January,February,March,April,May,June,July,August,September,October,November,December,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,2025年,2026年
0,300107150,BSSC-FLT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,300107150,FLT-REP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,300107150,T/S,0,0,0,0,0,0,0,0,0,0,0,0,60,20,80,70,80,20,19,27,25,30,45,30,1004,1421
3,300111590,BSSC-FLT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,300111590,FLT-REP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,TBR0L127,T/S,0,0,0,0,0,0,0,0,0,0,0,0,30,0,20,20,30,20,30,10,30,30,20,30,260,360
224,TBR0L132,BSSC-FLT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
225,TBR0L132,FLT-REP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
226,TBR0L132,T/S,0,0,0,0,0,0,0,0,0,0,0,0,80,30,30,30,30,40,40,40,60,65,50,45,490,670


In [245]:
## 讀取產品總表
pro_table = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\OB\24OB\24OB_product_table.csv")

In [254]:
pro_table["Rim"] = pro_table["Rim"].astype(str).map(lambda x: "22.5" if x == "225.0" else "19.5" if x == "195.0" else "17.5" if x == "175.0" else "16" if x == "16.0" else x)

In [260]:
pro_table["size"] = pro_table["Section"] + "R" + pro_table["Rim"]

In [265]:
pcode_dict = dict(zip(pro_table["PCODE"], pro_table[["size", "BP"]].apply(tuple, axis=1)))

In [268]:
final

,商品代號,通路明細,January,February,March,April,May,June,July,August,September,October,November,December,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,2025年,2026年
0,300107150,BSSC-FLT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,300107150,FLT-REP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,300107150,T/S,0,0,0,0,0,0,0,0,0,0,0,0,60,20,80,70,80,20,19,27,25,30,45,30,1004,1421
3,300111590,BSSC-FLT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,300111590,FLT-REP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,TBR0L127,T/S,0,0,0,0,0,0,0,0,0,0,0,0,30,0,20,20,30,20,30,10,30,30,20,30,260,360
224,TBR0L132,BSSC-FLT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
225,TBR0L132,FLT-REP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
226,TBR0L132,T/S,0,0,0,0,0,0,0,0,0,0,0,0,80,30,30,30,30,40,40,40,60,65,50,45,490,670


In [271]:
final["產品規格"] = final["商品代號"].map(pcode_dict)

In [282]:
final[["Size", "Pattern"]] = pd.DataFrame(final["產品規格"].tolist(), columns=["Size", "Pattern"])

In [286]:
final = final[['商品代號', '通路明細', 'Size', 'Pattern', 'January', 'February', 'March', 'April', 'May', 'June',
       'July', 'August', 'September', 'October', 'November', 'December', '1月',
       '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月',
       '2025年', '2026年']]

In [287]:
final.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\OB\24OB\24OB第一版數量總表格.xlsx", index=False)